In [0]:
%pip install openai==0.28

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
%pip install fitz

  Obtaining dependency information for fitz from https://files.pythonhosted.org/packages/7e/28/27f27d66eb82f24e6595deb26c0a875e62431878c416e38eac515023abb2/fitz-0.0.1.dev2-py2.py3-none-any.whl.metadata
  Obtaining dependency information for configobj from https://files.pythonhosted.org/packages/d3/bb/d10e531b297dd1d46f6b1fd11d018247af9f2d460037554bb7bb9011c6ac/configobj-5.0.8-py2.py3-none-any.whl.metadata
  Obtaining dependency information for nibabel from https://files.pythonhosted.org/packages/77/3f/ce43b8c2ccc4a7913a87c4d425aaf0080ea3abf947587e47dc2025981a17/nibabel-5.2.1-py3-none-any.whl.metadata
  Obtaining dependency information for nipype from https://files.pythonhosted.org/packages/60/02/d4e8c990e14d7efcb35038a3d1743ff5068d66d92c1fcc298c9654bae55d/nipype-1.8.6-py3-none-any.whl.metadata
  Obtaining dependency information for pyxnat from https://files.pythonhosted.org/packages/97/f6/dbe92707b35f3fec297228e81cbf35e0b1028e20c98a0cea1650e967f88a/pyxnat-1.6.2-py3-none-any.whl.metadat

In [0]:
%pip install PyMuPDF

  Obtaining dependency information for PyMuPDF from https://files.pythonhosted.org/packages/da/1e/80adebf57a61b270e54d2a626dfe33173d3779fca0b615833967f80ec6cd/PyMuPDF-1.24.9-cp311-none-manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for PyMuPDFb==1.24.9 from https://files.pythonhosted.org/packages/73/99/b54b666098c6085c19c6e9848430d39398f7fab3141de0873e7db358d6b3/PyMuPDFb-1.24.9-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/3.5 MB ? eta -:--:--
   ━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.3/3.5 MB 8.2 MB/s eta 0:00:01
   ━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.6/3.5 MB 8.8 MB/s eta 0:00:01
   ━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/3.5 MB 11.6 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━ 1.9/3.5 MB 14.4 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━ 3.0/3.5 MB 17.4 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 3.5/3.5 MB 18.7 MB/s eta 0:00:01
   

In [0]:
dbutils.library.restartPython()

In [0]:
import openai
import pandas as pd
import json
import os
import re
import fitz
import time

from pyspark.sql.types import *

##Variables transversales

Dado que la informacion referente a la conexion puede ser reutilizada, se optom por utilisar el servicio de Keyvault para asi garantizar la seguridad de las claves de acceso del servicio

In [0]:
#claves para hacer la conexion a chat-gpt 4.0
openai.api_type = "azure"
openai.api_key = dbutils.secrets.get(scope = 'Prueba-Scope', key = 'KeyApi-OpenAI')
openai.api_base = dbutils.secrets.get(scope = 'Prueba-Scope', key = 'UriOpenAI')
openai.api_version = "2023-05-15"

Se definio un punto de montaje hacia el un blob storage, esto con el objetivo de manejar las operaciones respetando las definiciones de la arquitectura medallion, esto provee distintas etapas de calidad sobre los datos.

In [0]:
path_blob_bronze = "/mnt/storage/Bronze/Actual"
path_blob_silver = "/mnt/storage/Silver/Actual"
path_blob_gold = "/mnt/storage/Gold/Actual"

##Extraccion del texto plano de los archivos PDF

Como primera instancia, a nivel de bronze, llegaran los archivos PDF que corresponden a las CV de los candidatos, en esta etapa se ingestan y mediante la libreria fitz se obtiene el texto plano que se encuentra en cada PDF, esto debido a que la API de OpenAI recibe como parametros de entrada texto plano.

In [0]:
#Esta funcion lee el txt ya procesado
def get_text(text_file):
  with open(text_file,'r') as f:
    text = f.read()
    f.close()
  return text

In [0]:
def clean_text(text):
    #Reemplazar saltos de línea y tabulaciones con un espacio
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'\t', ' ', text)
    
    #Reemplazar secuencias de múltiples espacios con un solo espacio
    text = re.sub(r' +', ' ', text)
    
    #Reemplazar otros caracteres no deseados
    text = text.replace('\xa0', ' ')
    
    #Se liminan cadenas en blanco al final del texto
    text = text.strip()
    
    return text

Se obtienen las respectivas rutas de cada archivo en funcion del punto de montaje en bronze, de esta forma se extraera el texto del mismo y sera guardado en la capa de silver para su posterior consumo.

In [0]:
pdf_files = dbutils.fs.ls(path_blob_bronze)
pdf_files = [file.path for file in pdf_files if file.path.endswith(".pdf")]
pdf_files = [file.replace("dbfs:/", "/dbfs/") for file in pdf_files]

In [0]:
for pdf_file in pdf_files:
    pdf_document = fitz.open(pdf_file)
    text = ""
    
    #Se extrae el texto de cada página
    for page_num in range(pdf_document.page_count):
        page = pdf_document.load_page(page_num)
        text += page.get_text()

    #Se llama una funcion de limpieza debido a los saltos linea y otros caracteres especiales
    cleaned_text = clean_text(text)
    
    #Se define el nombre del archivo, el cual sera el mismo que provenia del pdf pero con una extension txt
    txt_file_name = pdf_file.replace(".pdf", ".txt").split("/")[-1]

    #Se guarda el texto en el Blob Storage
    with open("/dbfs" + path_blob_silver+"/"+txt_file_name, "w") as txt_file:
        txt_file.write(cleaned_text)

##Analisis de las hojas de vida mediante la API de OpenAI

Una vez que se encuentran definidos los archivos de texto plano sobre la capa de silver, se ingestan los datos, luego se definio un conjunto de funciones con prompts personalizados y categorizados para maximizar la respuesta del modelo.

Cada funcion se encargara de obtener el valor solicitado a traves del JSON provisto por la API y posteriormente sera almacenado en una lista para luego generar el conjunto de datos de forma estructurada, de esta forma se podran ver con mayor facilidad todos los resultados de los candidatos.

In [0]:
#Obtener nombre del candidato
def get_nombre_candidato(text):
  user_input = f'''---
  Basado en el siguiente texto el cual corresponde a una hoja de vida o CV "{text}"
  ---
  Indica de forma especifica, ¿cual el nombre completo de la persona?'''
  system_prompt = f'''
  Tienes esta informacion y solo debes responder basado en esta informacion, ademas el texto de respuesta no debe ser extenso. La respuesta que debes dar es: 'El nombre es' : {text}
  '''
  messages=[
      {"role": "system", "content": system_prompt},
      {"role": "user", "content": user_input},
  ]
  response = openai.ChatCompletion.create(engine="demo-iwco", messages=messages, temperature=0)
  result = response.get('choices', [{}])[0].get('message', {}).get('content', None)
  return result

In [0]:
def get_numero(text):
  user_input = f'''---
  Basado en el siguiente texto el cual corresponde a una hoja de vida o CV "{text}"
  ---
  Indica de forma especifica, ¿cual el numero celular o de telefono de la persona?'''
  system_prompt = f'''
  Tienes esta informacion y solo debes responder basado en esta informacion. La respuesta que debes dar es: 'El numero de contacto es' : {text}
  '''
  messages=[
      {"role": "system", "content": system_prompt},
      {"role": "user", "content": user_input},
  ]
  response = openai.ChatCompletion.create(engine="demo-iwco", messages=messages, temperature=0)
  result = response.get('choices', [{}])[0].get('message', {}).get('content', None)
  return result

In [0]:
def years_experience(text):
  user_input = f'''---
  Basado en el siguiente texto el cual corresponde a una hoja de vida o CV "{text}"
  ---
  Indica de forma especifica, ¿cuantos años de experiencia tiene la persona?. En caso que no este explicita la cantidad de años, determina los años de experiencia en funcion de los periodos de tiempo trabajados por la persona'''
  system_prompt = f'''
  Tienes esta informacion y solo debes responder basado en esta informacion. La respuesta que debes dar es: 'Tiene un total de' : {text}
  '''
  messages=[
      {"role": "system", "content": system_prompt},
      {"role": "user", "content": user_input},
  ]
  response = openai.ChatCompletion.create(engine="demo-iwco", messages=messages, temperature=0)
  result = response.get('choices', [{}])[0].get('message', {}).get('content', None)
  return result

In [0]:
def formacion_IA(text):
  user_input = f'''---
  Basado en el siguiente texto el cual corresponde a una hoja de vida o CV "{text}"
  ---
  Indica de forma especifica, ¿la persona tiene alguna formacion en IA?.'''
  system_prompt = f'''
  Tienes esta informacion y solo debes responder basado en esta informacion. En caso de ser afirmativa la respuesta, debes indicar : 'Si, tiene conocimiento en las siguientes herramientas de IA' : {text}. Caso contrario, solo debes responder : 'No, no tiene conocimiento'.
  '''
  messages=[
      {"role": "system", "content": system_prompt},
      {"role": "user", "content": user_input},
  ]
  response = openai.ChatCompletion.create(engine="demo-iwco", messages=messages, temperature=0)
  result = response.get('choices', [{}])[0].get('message', {}).get('content', None)
  return result

In [0]:
#Esta funcion se encarga de llamar las consultas de busqueda definidas.

def get_fields(txt_file):
  data = [] 
  text = get_text(txt_file)

  nombre_candidato = get_nombre_candidato(text=text)
  numero_contacto = get_numero(text=text)
  experiencia = years_experience(text=text)
  formacion = formacion_IA(text=text)
  data.append((nombre_candidato,numero_contacto,experiencia,formacion))

  return data

In [0]:
#Se obtienen los archivos txt desde silver para poder extraer los valores solicitados de cada uno de ellos.
txt_files = dbutils.fs.ls(path_blob_silver)
txt_files = [file.path for file in txt_files if file.path.endswith(".txt")]
txt_files = [file.replace("dbfs:/", "/dbfs/") for file in txt_files]

In [0]:
all_results = []

for i in txt_files:
    file_results = get_fields(i)
    all_results.extend(file_results)

In [0]:
schema = StructType([
    StructField("Nombre_Candidato", StringType(), True),
    StructField("Numero_Contacto", StringType(), True),
    StructField("Años_Experiencia", StringType(), True),
    StructField("Experiencia_IA", StringType(), True)
])

df = spark.createDataFrame(all_results, schema=schema)

In [0]:
df.display()

Nombre_Candidato,Numero_Contacto,Años_Experiencia,Experiencia_IA
El nombre es: Junaedi Puji,El numero de contacto es: +6282331472499,Tiene un total de más de 6 años de experiencia.,"No, no tiene conocimiento."
El nombre es: Powell Finwood,El numero de contacto es: +123-456-7890,Tiene un total de 3 años y 10 meses de experiencia.,"No, no tiene conocimiento."
El nombre es: Michael Smith,El numero de contacto es: 'Email me on Indeed: indeed.com/r/falicent/140749dace5dc26f',Tiene un total de 10+ years of Experience.,"No, no tiene conocimiento."
El nombre es: Dyah Hediyati S.Kom,El numero de contacto es: +62 85287404232,Tiene un total de 3 años de experiencia en analizar y procesar datos de transacciones.,"No, no tiene conocimiento."
El nombre es: IMMANUEL ABRAHAM MAHARDHIKA YUDANTORO TOBING,El numero de contacto es: +62 8577 7124 773,Tiene un total de 6 años de experiencia.,"No, no tiene conocimiento."
El nombre es: Alice Clark,El numero de contacto es,Tiene un total de 20+ años de experiencia.,"Sí, tiene conocimiento en las siguientes herramientas de IA: Machine Learning, Natural Language Processing."
El nombre es: Ringgi Cahyo Dwiputra,El numero de contacto es: 085157115062,Tiene un total de 2 años y 8 meses de experiencia profesional.,"No, no tiene conocimiento."
El nombre es: DR.SANTOSH KAKADE,El número de contacto es: 09850671175 o 09422071490.,Tiene un total de 20 años de experiencia en atención al paciente y cirugía.,"No, no tiene conocimiento."
El nombre es Loren Shevitz.,El numero de contacto es: 773-665-1234,Tiene un total de 27 años de experiencia.,"No, no tiene conocimiento."


In [0]:
#Los datos resultantes ya estructurados se exportan a la capa de gold para un posible consumo
df.write.parquet(path_blob_gold+"/candidatos")